## Data modeling exercise

In [82]:
import os,re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.metrics import classification_report,confusion_matrix,log_loss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

pd.set_option('display.max_columns',200)
pd.set_option("display.max_rows",525)

In [112]:
dfTrain = pd.read_csv("exercise_40_train.csv")
dfTest = pd.read_csv("exercise_40_test.csv")


In [113]:
dfTrain.head(15)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100
0,0,0.165254,18.060003,Wed,1.077380,-1.339233,-1.584341,0.0062%,0.220784,1.816481,1.171788,109.626841,4.644568,4.814885,1.541740,-0.587361,101.906052,4.278444,77.709700,$-908.650758424405,7.328029,36.601967,126.384294,1.264713,female,3.834082,2.352406,905.491445,14.779467,0.752524,NaN,no,-0.597288,Colorado,46.667221,3.159002,69.444841,32.450076,NaN,5-10 miles,18.129499,NaN,0.904626,1.227266,0.703326,-0.810371,0.234654,53.359113,58.251700,16.084987,1602.756464,2.740134,7.617666,72.167186,0.166127,NaN,-1.532562,NaN,300.629990,0,August,9.840873,6.600008,1.252479,46.992716,farmers,1.212048,12.896733,15.263491,0.518653,0.543768,-0.912506,53.521544,7.723719,NaN,NaN,NaN,mercedes,51.287604,1.0,11.131462,-0.471594,-3.261865,0.085710,0.383351,126.935322,47.872599,-0.162668,1.079562,10.602395,2.770844,-0.397427,0.909479,no,5.492487,NaN,10.255579,7.627730,0,yes,104.251338
1,1,2.441471,18.416307,Friday,1.482586,0.920817,-0.759931,0.0064%,1.192441,3.513950,1.419900,84.079367,1.459868,1.443983,NaN,-1.165869,100.112121,8.487132,70.808566,$-1864.9622875143,6.796408,3.789176,93.037021,0.346201,male,4.345028,1.651579,868.784447,25.914680,1.445294,NaN,no,1.127376,Tennessee,47.928569,-1.013712,61.289132,35.521824,31.526217,5-10 miles,11.389081,99074.100620,-1.157897,-0.822055,-0.851141,2.651773,1.104631,37.728834,56.052749,NaN,3350.837875,1.995635,NaN,73.964560,NaN,NaN,0.956001,NaN,300.629990,0,April,6.939395,3.864254,0.057936,48.216622,allstate,0.738526,14.402071,33.940951,-0.140384,1.016703,-0.000234,46.797571,7.260365,-1.731529,0.666354,-2.870800,mercedes,42.918352,NaN,11.715043,1.691428,-4.789183,4.684722,2.138771,102.409522,46.584780,1.252022,1.223852,10.408146,3.703013,0.656651,9.093466,no,3.346429,4.321172,NaN,10.505284,1,yes,101.230645
2,1,4.427278,19.188092,Thursday,0.145652,0.366093,0.709962,-8e-04%,0.952323,0.782974,-1.247022,95.375221,1.098525,1.216059,0.450624,0.211685,99.215210,8.601193,75.922820,$-543.187402955527,3.802395,7.407996,71.022413,0.070968,male,4.332644,-0.375737,1077.233497,13.177479,4.174536,NaN,no,2.605279,Texas,56.674425,0.108486,69.798923,30.684074,31.049447,5-10 miles,14.372443,100087.339539,0.869508,0.150728,NaN,-0.856591,-2.561083,50.236892,63.975108,6.998154,3756.910196,1.772648,15.057485,59.428690,1.844493,NaN,4.127857,NaN,182.369349,0,September,6.228138,1.370661,-0.239705,54.120933,geico,-0.032980,14.402071,18.211817,-0.819798,1.010811,-0.000234,48.202036,9.336021,0.209817,1.124866,-3.558718,subaru,55.020333,1.0,5.660882,-2.608974,-6.387984,2.506272,2.138771,NaN,53.977291,0.657292,-0.353469,NaN,3.997183,2.059615,0.305170,no,4.456565,NaN,8.754572,7.810979,0,yes,109.345215
3,0,3.925235,19.901257,Tuesday,1.763602,-0.251926,-0.827461,-0.0057%,-0.520756,1.825586,2.223038,96.420382,-1.390239,3.962961,NaN,-2.046856,NaN,6.611554,74.966925,$-182.626380634258,7.728963,8.136213,121.610846,0.700954,male,7.294990,-0.603983,1051.655489,17.006528,2.347355,NaN,no,1.071202,Minnesota,59.154933,1.319711,65.408246,34.401290,48.363690,5-10 miles,13.191173,100934.096543,NaN,-0.965711,NaN,0.422522,-2.123048,41.857197,59.226119,NaN,1961.609788,3.155214,NaN,68.671023,-1.020225,5.833712,0.663759,NaN,300.629990,0,September,6.005140,0.013162,0.318335,54.784192,geico,-0.466535,14.402071,14.629914,1.389325,0.704880,-1.510949,49.882647,5.661421,1.606797,1.726010,-0.398417,nissan,47.769343,1.0,7.472328,1.424316,-5.431998,3.285291,2.138771,105.208424,49.543472,2.066346,1.761266,NaN,2.933707,0.899392,5.971782,no,4.100022,1.151085,NaN,9.178325,1,yes,103.021970
4,0,2.868802,22.202473,Sunday,3.405119,0.083162,1.381504,0.0109%,-0.732739,2.151990,-0.275406,90.76

## Merge data

In [114]:
dfCombine = pd.concat([dfTrain,dfTest])
dfCombine.iloc[40000:40001,]


,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100
0,NaN,4.747627,20.509439,Wednesday,2.299105,-1.815777,-0.752166,0.0098%,-3.240309,0.587948,-0.260721,101.113628,-0.812035,3.251085,-0.004432,-0.917763,102.193597,7.097387,75.256714,$120.216189955777,8.269754,4.794438,56.134458,-0.083641,NaN,5.740955,-3.15254,924.013304,17.697125,1.402273,NaN,no,1.461654,Florida,57.930285,4.7273,66.416594,28.450466,33.49931,5-10 miles,16.776061,99971.844703,0.492812,-0.963616,NaN,NaN,1.266416,53.020326,61.840284,NaN,1825.413159,2.517924,NaN,70.880778,NaN,1.923125,0.239009,NaN,300.62999,0,May,5.461123,5.149473,1.179229,59.346822,progressive,2.219502,17.667792,5.557066,-2.030901,0.636111,-0.000234,62.363381,4.613878,2.415655,3.632041,-0.382482,mercedes,55.938387,1.0,8.325299,-1.274085,-5.663245,2.40266,-0.06104,NaN,64.0025,0.548974,3.420875,11.553023,5.904644,NaN,12.542333,no,3.107683,0.533904,12.438759,7.298306,0,NaN,93.56712


In [115]:
dfCombine.reset_index(drop=True,inplace=True)

In [116]:
dfCombine.tail(15)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100
49985,NaN,-0.736014,20.521246,Mon,-1.063750,0.026207,1.213590,-0.0147%,1.259334,3.772195,0.027704,NaN,1.676515,5.121585,NaN,-0.938329,NaN,9.051384,75.721721,$2025.04351452311,0.490527,1.169093,57.398798,-0.146897,female,7.059847,1.319189,1106.036434,13.169400,0.953751,NaN,yes,0.573764,Oregon,53.539255,-0.822446,73.705643,25.750056,NaN,5-10 miles,10.067769,98726.755725,2.669192,2.947775,NaN,0.441242,-1.128505,52.121453,62.488409,NaN,1631.531367,3.632953,NaN,66.526842,-0.060789,1.457902,-0.385146,26.684241,300.629990,1,June,NaN,2.742502,-0.030588,50.850863,esurance,-0.877116,14.402071,NaN,0.824507,1.062483,-0.000234,50.965109,4.927721,1.474674,-0.582461,0.620936,NaN,NaN,1.0,13.007662,1.100808,-4.433106,2.493231,2.138771,103.862650,48.523133,-2.505763,0.497260,NaN,1.512272,0.747831,-0.425794,no,0.959616,0.354410,9.987922,10.878594,0,yes,97.811463
49986,NaN,3.712521,19.165130,Sunday,1.830575,0.453073,-1.032832,0.0033%,0.028534,2.666954,0.917458,102.242967,6.576326,6.532160,0.602609,1.340312,99.463272,9.779164,71.647983,$551.556027153297,4.728445,13.650722,33.534932,-0.679683,male,6.992263,1.341141,946.456346,19.416014,3.081420,-0.118341,no,0.522947,California,51.070986,1.629298,67.913747,27.362753,18.876847,5-10 miles,10.450876,98364.762399,-1.103506,1.864032,NaN,-0.376934,1.553511,43.217426,58.747941,8.105483,2067.160150,0.757456,NaN,69.331921,0.523956,0.439878,0.151020,NaN,363.160054,0,July,10.288280,3.243170,NaN,53.454268,allstate,1.207612,14.402071,20.722122,-1.663337,0.937230,-0.000234,46.493147,6.659511,-0.166853,-0.107964,NaN,buick,44.252793,1.0,13.685365,0.795794,-5.596558,2.551467,2.138771,NaN,53.800101,-1.463127,0.257929,10.024261,5.191872,0.130769,6.536151,no,5.920077,2.766128,8.730658,10.663369,0,NaN,95.004361
49987,NaN,0.709832,16.587671,Sat,-1.800409,-1.028227,1.118936,-0.0066%,1.101568,4.337200,0.801664,104.898301,0.750542,4.082391,NaN,1.164066,99.298691,11.030309,75.748940,$-317.180512987988,4.399880,0.029930,90.708540,1.751903,male,7.113489,-2.448262,1048.539708,14.207746,-2.467190,NaN,no,0.640640,South Carolina,52.978748,2.124352,69.740660,35.805583,29.242704,5-10 miles,11.712159,99309.392489,NaN,-0.328943,NaN,1.697430,0.226162,50.518948,65.373842,NaN,3129.347342,0.439898,13.811304,62.123992,-1.056065,NaN,0.831958,NaN,300.629990,0,December,7.456765,5.465142,NaN,54.951326,esurance,-0.723682,14.402071,21.846048,0.760048,1.238322,-0.000234,52.768163,14.128794,0.606747,2.399640,-2.063285,ford,46.921408,1.0,9.159948,-1.574925,-6.321939,2.341064,2.138771,106.578659,45.932345,0.698349,-1.080365,NaN,3.225801,-0.022385,6.514797,no,4.090027,NaN,10.494749,9.508210,1,yes,106.657515
49988,NaN,2.334473,21.244453,Sun,-0.723619,-0.875640,-1.765345,0.0115%,0.892667,2.059836,0.377693,93.963936,0.461947,3.426426,NaN,-0.009906,100.784775,9.188568,72.209640,$-806.027507326278,1.582737,4.426977,40.325717,0.848079,NaN,1.928928,0.645825,1074.744838,17.416748,5.167677,NaN,no,4.238717,NaN,54.379896,0.481625,70.707886,25.567279,84.847037,5-10 miles,11.871436,NaN,0.781929,0.466086,0.766055,0.018412,0.466561,49.440586,70.257386,NaN,3284.701611,-1.468251,5.534820,62.707036,NaN,4.146522,0.253623,NaN,300.629990,0,January,4.698075,3.382028,1.776860,59.306625,allstate,0.024762,14.402071,23.061928,0.146350,1.625178,-0.000234,54.300211,4.479601,NaN,3.038046,-0.109401,ford,49.978955,1.0,8.752467,-0.447530,-6.405434,1.052414,2.138771,98.724511,53.189115,0.505254,0.110240,11.175302,2.400006,0.932031,1.818764,no,3.267682,-0.266084,8.320911,9.146805,0,NaN,102.901152
49989,NaN,0.246916,18.310140,Wed,1.690758,0.423054,-1.224011,-0.0147%,2.022091,2.944178,-0.443466,106

In [117]:
features = ['y',
 'x1',
 'x2',
 'x3',
 'x4',
 'x5',
 'x6',
 'x7',
 'x8',
 'x9',
 'x10',
 'x11',
 'x12',
 'x13',
 'x14',
 'x15',
 'x16',
 'x17',
 'x18',
 'x19',
 'x20',
 'x21',
 'x22',
 'x23',
 'x24',
 'x25',
 'x26',
 'x27',
 'x28',
 'x29',
 'x30',
 'x31',
 'x32',
 'x33',
 'x34',
 'x35',
 'x36',
 'x37',
 'x38',
 'x39',
 'x40',
 'x41',
 'x42',
 'x43',
 'x44',
 'x45',
 'x46',
 'x47',
 'x48',
 'x49',
 'x50',
 'x51',
 'x52',
 'x53',
 'x54',
 'x55',
 'x56',
 'x57',
 'x58',
 'x59',
 'x60',
 'x61',
 'x62',
 'x63',
 'x64',
 'x65',
 'x66',
 'x67',
 'x68',
 'x69',
 'x70',
 'x71',
 'x72',
 'x73',
 'x74',
 'x75',
 'x76',
 'x77',
 'x78',
 'x79',
 'x80',
 'x81',
 'x82',
 'x83',
 'x84',
 'x85',
 'x86',
 'x87',
 'x88',
 'x89',
 'x90',
 'x91',
 'x92',
 'x93',
 'x94',
 'x95',
 'x96',
 'x97',
 'x98',
 'x99',
 'x100',
 'map_days_x3',
 'encode_gender_x24',
 'encode_x31',
 'cat_days_x3_Friday',
 'cat_days_x3_Monday',
 'cat_days_x3_Saturday',
 'cat_days_x3_Sunday',
 'cat_days_x3_Thursday',
 'cat_days_x3_Tuesday',
 'cat_days_x3_Wednesday',
 'cat_insurance_x65_allstate',
 'cat_insurance_x65_esurance',
 'cat_insurance_x65_farmers',
 'cat_insurance_x65_geico',
 'cat_insurance_x65_progressive',
 'cat_state_x33_Alabama',
 'cat_state_x33_Alaska',
 'cat_state_x33_Arizona',
 'cat_state_x33_Arkansas',
 'cat_state_x33_California',
 'cat_state_x33_Colorado',
 'cat_state_x33_Connecticut',
 'cat_state_x33_DC',
 'cat_state_x33_Delaware',
 'cat_state_x33_Florida',
 'cat_state_x33_Georgia',
 'cat_state_x33_Hawaii',
 'cat_state_x33_Idaho',
 'cat_state_x33_Illinois',
 'cat_state_x33_Indiana',
 'cat_state_x33_Iowa',
 'cat_state_x33_Kansas',
 'cat_state_x33_Kentucky',
 'cat_state_x33_Louisiana',
 'cat_state_x33_Maine',
 'cat_state_x33_Maryland',
 'cat_state_x33_Massachusetts',
 'cat_state_x33_Michigan',
 'cat_state_x33_Minnesota',
 'cat_state_x33_Mississippi',
 'cat_state_x33_Missouri',
 'cat_state_x33_Montana',
 'cat_state_x33_Nebraska',
 'cat_state_x33_Nevada',
 'cat_state_x33_New Hampshire',
 'cat_state_x33_New Jersey',
 'cat_state_x33_New Mexico',
 'cat_state_x33_New York',
 'cat_state_x33_North Carolina',
 'cat_state_x33_North Dakota',
 'cat_state_x33_Ohio',
 'cat_state_x33_Oklahoma',
 'cat_state_x33_Oregon',
 'cat_state_x33_Pennsylvania',
 'cat_state_x33_Rhode Island',
 'cat_state_x33_South Carolina',
 'cat_state_x33_South Dakota',
 'cat_state_x33_Tennessee',
 'cat_state_x33_Texas',
 'cat_state_x33_Utah',
 'cat_state_x33_Vermont',
 'cat_state_x33_Virginia',
 'cat_state_x33_Washington',
 'cat_state_x33_West Virginia',
 'cat_state_x33_Wisconsin',
 'cat_state_x33_Wyoming',
 'cat_car_manufacturer_x77_buick',
 'cat_car_manufacturer_x77_chevrolet',
 'cat_car_manufacturer_x77_ford',
 'cat_car_manufacturer_x77_mercedes',
 'cat_car_manufacturer_x77_nissan',
 'cat_car_manufacturer_x77_subaru',
 'cat_car_manufacturer_x77_toyota']


## Restrict data set

In [118]:
dfFilter = dfCombine[(dfCombine["y"].isnull() == False)]
dfFilter = dfFilter[(dfFilter["x5"].isnull() == False)]
dfFilter = dfFilter[(dfFilter["x26"].isnull() == False)]
dfFilter = dfFilter[(dfFilter["x33"].isnull() == False)]
dfFilter = dfFilter[(dfFilter["x63"].isnull() == False)]
dfFilter = dfFilter[(dfFilter["x22"].isnull() == False)]
dfFilter = dfFilter[(dfFilter["x38"].isnull() == False)]
dfFilter = dfFilter[(dfFilter["x67"].isnull() == False)]
dfFilter = dfFilter[(dfFilter["x68"].isnull() == False)]
dfFilter = dfFilter[(dfFilter["x79"].isnull() == False)]
dfFilter = dfFilter[(dfFilter["x83"].isnull() == False)]
dfFilter = dfFilter[(dfFilter["x86"].isnull() == False)]
dfFilter = dfFilter[(dfFilter["x88"].isnull() == False)]
dfFilter = dfFilter[(dfFilter["x92"].isnull() == False)]
dfFilter = dfFilter[(dfFilter["x94"].isnull() == False)]
dfFilter = dfFilter[(dfFilter["x24"].isnull() == False)]
dfFilter = dfFilter[(dfFilter["x77"].isnull() == False)]


In [119]:
print(len(dfCombine))
print(len(dfFilter))

50000
10307


In [120]:
dfFilter.isnull().sum()

y          0
x1         0
x2         0
x3         0
x4         0
x5         0
x6         0
x7         0
x8         0
x9         0
x10        0
x11     1300
x12        0
x13        0
x14     2476
x15        0
x16     2930
x17        0
x18        0
x19        0
x20        0
x21        0
x22        0
x23        0
x24        0
x25        0
x26        0
x27        0
x28        0
x29        0
x30     8386
x31        0
x32        0
x33        0
x34        0
x35        0
x36        0
x37        0
x38        0
x39        0
x40        0
x41     2438
x42     2439
x43        0
x44     8829
x45     2053
x46        0
x47        0
x48        0
x49     3312
x50        0
x51        0
x52     4200
x53        0
x54     3205
x55     4584
x56        0
x57     8379
x58        0
x59        0
x60        0
x61     1877
x62        0
x63        0
x64     1335
x65        0
x66        0
x67        0
x68        0
x69        0
x70        0
x71        0
x72        0
x73        0
x74     3445
x75     1389
x76     1367

In [121]:
dfFilter["x3"].unique()

array(['Thursday', 'Tuesday', 'Sunday', 'Tue', 'Fri', 'Monday',
       'Saturday', 'Thur', 'Sun', 'Wednesday', 'Friday', 'Mon', 'Wed',
       'Sat'], dtype=object)

In [122]:
dfFilter["x24"].unique()

array(['male', 'female'], dtype=object)

In [123]:
dfFilter["x31"].unique()

array(['no', 'yes'], dtype=object)

In [124]:
dfFilter["x33"].unique()

array(['Texas', 'Minnesota', 'New York', 'Florida', 'Pennsylvania',
       'Iowa', 'Vermont', 'Ohio', 'Georgia', 'Virginia', 'Washington',
       'California', 'Missouri', 'Oregon', 'Illinois', 'Utah',
       'Wisconsin', 'Indiana', 'New Jersey', 'Arizona', 'North Dakota',
       'Louisiana', 'Tennessee', 'Alabama', 'Maryland', 'Massachusetts',
       'Alaska', 'North Carolina', 'Kentucky', 'New Mexico', 'Wyoming',
       'Michigan', 'Colorado', 'Delaware', 'New Hampshire', 'Nebraska',
       'Idaho', 'Rhode Island', 'Mississippi', 'Kansas', 'West Virginia',
       'Nevada', 'South Carolina', 'Arkansas', 'Hawaii', 'Connecticut',
       'Oklahoma', 'Maine', 'South Dakota', 'Montana', 'DC'], dtype=object)

In [125]:
dfFilter["x65"].unique()

array(['geico', 'farmers', 'esurance', 'allstate', 'progressive'],
      dtype=object)

In [126]:
dfFilter["x77"].unique()

array(['subaru', 'nissan', 'toyota', 'mercedes', 'ford', 'chevrolet',
       'buick'], dtype=object)

In [127]:
dfFilter.head(20)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100
2,1.0,4.427278,19.188092,Thursday,0.145652,0.366093,0.709962,-8e-04%,0.952323,0.782974,-1.247022,95.375221,1.098525,1.216059,0.450624,0.211685,99.215210,8.601193,75.922820,$-543.187402955527,3.802395,7.407996,71.022413,0.070968,male,4.332644,-0.375737,1077.233497,13.177479,4.174536,NaN,no,2.605279,Texas,56.674425,0.108486,69.798923,30.684074,31.049447,5-10 miles,14.372443,100087.339539,0.869508,0.150728,NaN,-0.856591,-2.561083,50.236892,63.975108,6.998154,3756.910196,1.772648,15.057485,59.428690,1.844493,NaN,4.127857,NaN,182.369349,0,September,6.228138,1.370661,-0.239705,54.120933,geico,-0.032980,14.402071,18.211817,-0.819798,1.010811,-0.000234,48.202036,9.336021,0.209817,1.124866,-3.558718,subaru,55.020333,1.0,5.660882,-2.608974,-6.387984,2.506272,2.138771,NaN,53.977291,0.657292,-0.353469,NaN,3.997183,2.059615,0.305170,no,4.456565,NaN,8.754572,7.810979,0,yes,109.345215
3,0.0,3.925235,19.901257,Tuesday,1.763602,-0.251926,-0.827461,-0.0057%,-0.520756,1.825586,2.223038,96.420382,-1.390239,3.962961,NaN,-2.046856,NaN,6.611554,74.966925,$-182.626380634258,7.728963,8.136213,121.610846,0.700954,male,7.294990,-0.603983,1051.655489,17.006528,2.347355,NaN,no,1.071202,Minnesota,59.154933,1.319711,65.408246,34.401290,48.363690,5-10 miles,13.191173,100934.096543,NaN,-0.965711,NaN,0.422522,-2.123048,41.857197,59.226119,NaN,1961.609788,3.155214,NaN,68.671023,-1.020225,5.833712,0.663759,NaN,300.629990,0,September,6.005140,0.013162,0.318335,54.784192,geico,-0.466535,14.402071,14.629914,1.389325,0.704880,-1.510949,49.882647,5.661421,1.606797,1.726010,-0.398417,nissan,47.769343,1.0,7.472328,1.424316,-5.431998,3.285291,2.138771,105.208424,49.543472,2.066346,1.761266,NaN,2.933707,0.899392,5.971782,no,4.100022,1.151085,NaN,9.178325,1,yes,103.021970
4,0.0,2.868802,22.202473,Sunday,3.405119,0.083162,1.381504,0.0109%,-0.732739,2.151990,-0.275406,90.769952,7.230125,3.877312,0.392002,-1.201565,100.626558,9.103015,77.977156,$967.007090837503,2.272765,24.452102,56.492317,-1.102387,male,6.313827,0.429187,949.904947,16.962710,0.510985,NaN,yes,2.283921,New York,46.445617,0.022747,66.662910,31.135261,31.819899,5-10 miles,17.210430,NaN,NaN,1.012093,NaN,-0.348240,3.477451,47.844153,55.921988,NaN,2345.195505,3.253079,14.193721,71.853326,0.926779,4.115990,-2.273628,NaN,149.725023,0,January,7.654926,1.305936,0.418272,51.486405,geico,-0.195764,14.402071,12.227512,-2.951163,1.096548,-0.000234,51.349106,9.422401,3.488398,1.884259,0.019803,toyota,44.640410,1.0,6.530625,0.705003,-5.664815,3.395916,2.138771,96.150945,54.843346,0.663113,-0.952377,NaN,2.922302,3.003595,1.046096,yes,3.234033,2.074927,9.987006,11.702664,0,yes,92.925935
13,0.0,3.671553,22.207880,Tue,0.698724,-1.862047,2.153832,0.0068%,0.205453,1.309530,0.683070,111.472466,1.433037,2.625811,0.313683,-0.824623,102.816034,6.134170,72.430917,$-377.495529325022,7.074143,7.676579,72.703358,0.520186,male,0.849677,1.178534,759.765566,17.523317,0.444232,-1.549220,no,1.130353,Florida,41.982861,2.309857,68.489433,31.802446,29.893769,5-10 miles,16.987226,97628.442038,NaN,0.440045,NaN,0.672879,3.121838,50.335336,55.979128,9.887514,1564.639941,3.842460,NaN,73.728585,1.366928,NaN,0.186035,47.142610,300.629990,0,January,13.048572,8.884421,1.097294,44.834865,farmers,-0.211141,14.402071,26.651846,1.319336,0.651186,-0.000234,60.552217,9.469803,-0.989563,0.218994,0.951720,mercedes,44.449820,1.0,14.601537,1.425902,-3.274212,3.601828,2.138771,93.228245,55.869454,0.612429,0.587316,11.003700,2.347867,1.458799,4.470832,no,3.482305,0.951736,9.245415,10.542323,0,yes,95.249052
19,0.0,1.085729,20.656173,Fri,2.082224,-1.852744,-0.927487,-0.0086%,-0.2

## Map days, gender, & x31

In [128]:
# dfFilter["x3"] = dfFilter["x3"].map({"Mon":"Monday","Tue":"Tuesday","Wed":"Wednesday", \
#     "Thur":"Thursday","Fri":"Friday",'Sat':'Saturday','Sun':'Sunday'})

dfFilter["map_days_x3"] = dfFilter["x3"].replace({"Mon":"Monday","Tue":"Tuesday","Wed":"Wednesday", \
    "Thur":"Thursday","Fri":"Friday",'Sat':'Saturday','Sun':'Sunday'})

In [129]:
dfFilter["encode_gender_x24"] = dfFilter["x24"].map({"male":0,"female":1})

In [130]:
dfFilter["encode_gender_x24"].unique()

array([0, 1])

In [131]:
dfFilter["encode_x31"] = dfFilter["x31"].map({"no":0,"yes":1})

In [132]:
dfFilter.head(15)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,map_days_x3,encode_gender_x24,encode_x31
2,1.0,4.427278,19.188092,Thursday,0.145652,0.366093,0.709962,-8e-04%,0.952323,0.782974,-1.247022,95.375221,1.098525,1.216059,0.450624,0.211685,99.215210,8.601193,75.922820,$-543.187402955527,3.802395,7.407996,71.022413,0.070968,male,4.332644,-0.375737,1077.233497,13.177479,4.174536,NaN,no,2.605279,Texas,56.674425,0.108486,69.798923,30.684074,31.049447,5-10 miles,14.372443,100087.339539,0.869508,0.150728,NaN,-0.856591,-2.561083,50.236892,63.975108,6.998154,3756.910196,1.772648,15.057485,59.428690,1.844493,NaN,4.127857,NaN,182.369349,0,September,6.228138,1.370661,-0.239705,54.120933,geico,-0.032980,14.402071,18.211817,-0.819798,1.010811,-0.000234,48.202036,9.336021,0.209817,1.124866,-3.558718,subaru,55.020333,1.0,5.660882,-2.608974,-6.387984,2.506272,2.138771,NaN,53.977291,0.657292,-0.353469,NaN,3.997183,2.059615,0.305170,no,4.456565,NaN,8.754572,7.810979,0,yes,109.345215,Thursday,0,0
3,0.0,3.925235,19.901257,Tuesday,1.763602,-0.251926,-0.827461,-0.0057%,-0.520756,1.825586,2.223038,96.420382,-1.390239,3.962961,NaN,-2.046856,NaN,6.611554,74.966925,$-182.626380634258,7.728963,8.136213,121.610846,0.700954,male,7.294990,-0.603983,1051.655489,17.006528,2.347355,NaN,no,1.071202,Minnesota,59.154933,1.319711,65.408246,34.401290,48.363690,5-10 miles,13.191173,100934.096543,NaN,-0.965711,NaN,0.422522,-2.123048,41.857197,59.226119,NaN,1961.609788,3.155214,NaN,68.671023,-1.020225,5.833712,0.663759,NaN,300.629990,0,September,6.005140,0.013162,0.318335,54.784192,geico,-0.466535,14.402071,14.629914,1.389325,0.704880,-1.510949,49.882647,5.661421,1.606797,1.726010,-0.398417,nissan,47.769343,1.0,7.472328,1.424316,-5.431998,3.285291,2.138771,105.208424,49.543472,2.066346,1.761266,NaN,2.933707,0.899392,5.971782,no,4.100022,1.151085,NaN,9.178325,1,yes,103.021970,Tuesday,0,0
4,0.0,2.868802,22.202473,Sunday,3.405119,0.083162,1.381504,0.0109%,-0.732739,2.151990,-0.275406,90.769952,7.230125,3.877312,0.392002,-1.201565,100.626558,9.103015,77.977156,$967.007090837503,2.272765,24.452102,56.492317,-1.102387,male,6.313827,0.429187,949.904947,16.962710,0.510985,NaN,yes,2.283921,New York,46.445617,0.022747,66.662910,31.135261,31.819899,5-10 miles,17.210430,NaN,NaN,1.012093,NaN,-0.348240,3.477451,47.844153,55.921988,NaN,2345.195505,3.253079,14.193721,71.853326,0.926779,4.115990,-2.273628,NaN,149.725023,0,January,7.654926,1.305936,0.418272,51.486405,geico,-0.195764,14.402071,12.227512,-2.951163,1.096548,-0.000234,51.349106,9.422401,3.488398,1.884259,0.019803,toyota,44.640410,1.0,6.530625,0.705003,-5.664815,3.395916,2.138771,96.150945,54.843346,0.663113,-0.952377,NaN,2.922302,3.003595,1.046096,yes,3.234033,2.074927,9.987006,11.702664,0,yes,92.925935,Sunday,0,1
13,0.0,3.671553,22.207880,Tue,0.698724,-1.862047,2.153832,0.0068%,0.205453,1.309530,0.683070,111.472466,1.433037,2.625811,0.313683,-0.824623,102.816034,6.134170,72.430917,$-377.495529325022,7.074143,7.676579,72.703358,0.520186,male,0.849677,1.178534,759.765566,17.523317,0.444232,-1.549220,no,1.130353,Florida,41.982861,2.309857,68.489433,31.802446,29.893769,5-10 miles,16.987226,97628.442038,NaN,0.440045,NaN,0.672879,3.121838,50.335336,55.979128,9.887514,1564.639941,3.842460,NaN,73.728585,1.366928,NaN,0.186035,47.142610,300.629990,0,January,13.048572,8.884421,1.097294,44.834865,farmers,-0.211141,14.402071,26.651846,1.319336,0.651186,-0.000234,60.552217,9.469803,-0.989563,0.218994,0.951720,mercedes,44.449820,1.0,14.601537,1.425902,-3.274212,3.601828,2.138771,93.228245,55.869454,0.612429,0.587316,11.003700,2.347867,1.458799,4.470832,no,3.482305,0.951736,9.245415,10.542323,0,yes,95.24

In [133]:
print(len(dfCombine))
print(len(dfFilter))

50000
10307


## Get dummies

In [134]:
catDaysX3 = pd.get_dummies(dfFilter["map_days_x3"],prefix='cat_days_x3')
dfFilter = pd.merge(
    left=dfFilter,
    right=catDaysX3,
    left_index=True,
    right_index=True,
)

dfFilter.head(15)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,map_days_x3,encode_gender_x24,encode_x31,cat_days_x3_Friday,cat_days_x3_Monday,cat_days_x3_Saturday,cat_days_x3_Sunday,cat_days_x3_Thursday,cat_days_x3_Tuesday,cat_days_x3_Wednesday
2,1.0,4.427278,19.188092,Thursday,0.145652,0.366093,0.709962,-8e-04%,0.952323,0.782974,-1.247022,95.375221,1.098525,1.216059,0.450624,0.211685,99.215210,8.601193,75.922820,$-543.187402955527,3.802395,7.407996,71.022413,0.070968,male,4.332644,-0.375737,1077.233497,13.177479,4.174536,NaN,no,2.605279,Texas,56.674425,0.108486,69.798923,30.684074,31.049447,5-10 miles,14.372443,100087.339539,0.869508,0.150728,NaN,-0.856591,-2.561083,50.236892,63.975108,6.998154,3756.910196,1.772648,15.057485,59.428690,1.844493,NaN,4.127857,NaN,182.369349,0,September,6.228138,1.370661,-0.239705,54.120933,geico,-0.032980,14.402071,18.211817,-0.819798,1.010811,-0.000234,48.202036,9.336021,0.209817,1.124866,-3.558718,subaru,55.020333,1.0,5.660882,-2.608974,-6.387984,2.506272,2.138771,NaN,53.977291,0.657292,-0.353469,NaN,3.997183,2.059615,0.305170,no,4.456565,NaN,8.754572,7.810979,0,yes,109.345215,Thursday,0,0,0,0,0,0,1,0,0
3,0.0,3.925235,19.901257,Tuesday,1.763602,-0.251926,-0.827461,-0.0057%,-0.520756,1.825586,2.223038,96.420382,-1.390239,3.962961,NaN,-2.046856,NaN,6.611554,74.966925,$-182.626380634258,7.728963,8.136213,121.610846,0.700954,male,7.294990,-0.603983,1051.655489,17.006528,2.347355,NaN,no,1.071202,Minnesota,59.154933,1.319711,65.408246,34.401290,48.363690,5-10 miles,13.191173,100934.096543,NaN,-0.965711,NaN,0.422522,-2.123048,41.857197,59.226119,NaN,1961.609788,3.155214,NaN,68.671023,-1.020225,5.833712,0.663759,NaN,300.629990,0,September,6.005140,0.013162,0.318335,54.784192,geico,-0.466535,14.402071,14.629914,1.389325,0.704880,-1.510949,49.882647,5.661421,1.606797,1.726010,-0.398417,nissan,47.769343,1.0,7.472328,1.424316,-5.431998,3.285291,2.138771,105.208424,49.543472,2.066346,1.761266,NaN,2.933707,0.899392,5.971782,no,4.100022,1.151085,NaN,9.178325,1,yes,103.021970,Tuesday,0,0,0,0,0,0,0,1,0
4,0.0,2.868802,22.202473,Sunday,3.405119,0.083162,1.381504,0.0109%,-0.732739,2.151990,-0.275406,90.769952,7.230125,3.877312,0.392002,-1.201565,100.626558,9.103015,77.977156,$967.007090837503,2.272765,24.452102,56.492317,-1.102387,male,6.313827,0.429187,949.904947,16.962710,0.510985,NaN,yes,2.283921,New York,46.445617,0.022747,66.662910,31.135261,31.819899,5-10 miles,17.210430,NaN,NaN,1.012093,NaN,-0.348240,3.477451,47.844153,55.921988,NaN,2345.195505,3.253079,14.193721,71.853326,0.926779,4.115990,-2.273628,NaN,149.725023,0,January,7.654926,1.305936,0.418272,51.486405,geico,-0.195764,14.402071,12.227512,-2.951163,1.096548,-0.000234,51.349106,9.422401,3.488398,1.884259,0.019803,toyota,44.640410,1.0,6.530625,0.705003,-5.664815,3.395916,2.138771,96.150945,54.843346,0.663113,-0.952377,NaN,2.922302,3.003595,1.046096,yes,3.234033,2.074927,9.987006,11.702664,0,yes,92.925935,Sunday,0,1,0,0,0,1,0,0,0
13,0.0,3.671553,22.207880,Tue,0.698724,-1.862047,2.153832,0.0068%,0.205453,1.309530,0.683070,111.472466,1.433037,2.625811,0.313683,-0.824623,102.816034,6.134170,72.430917,$-377.495529325022,7.074143,7.676579,72.703358,0.520186,male,0.849677,1.178534,759.765566,17.523317,0.444232,-1.549220,no,1.130353,Florida,41.982861,2.309857,68.489433,31.802446,29.893769,5-10 miles,16.987226,97628.442038,NaN,0.440045,NaN,0.672879,3.121838,50.335336,55.979128,9.887514,1564.639941,3.842460,NaN,73.728585,1.366928,NaN,0.186035,47.142610,300.629990,0,January,13.048572,8.884421,1.097294,44.834865,farmers,-0.211141,14.402071,26.651846,1.319336,0.651186,-0.000234,60.552217,9.469803,-0.989563,0.218994,0.951720,mercedes,44.4

In [135]:
catInsuranceX65 = pd.get_dummies(dfFilter["x65"],prefix="cat_insurance_x65")
dfFilter = pd.merge(
    left=dfFilter,
    right=catInsuranceX65,
    left_index=True,
    right_index=True,
)

dfFilter.head(15)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,map_days_x3,encode_gender_x24,encode_x31,cat_days_x3_Friday,cat_days_x3_Monday,cat_days_x3_Saturday,cat_days_x3_Sunday,cat_days_x3_Thursday,cat_days_x3_Tuesday,cat_days_x3_Wednesday,cat_insurance_x65_allstate,cat_insurance_x65_esurance,cat_insurance_x65_farmers,cat_insurance_x65_geico,cat_insurance_x65_progressive
2,1.0,4.427278,19.188092,Thursday,0.145652,0.366093,0.709962,-8e-04%,0.952323,0.782974,-1.247022,95.375221,1.098525,1.216059,0.450624,0.211685,99.215210,8.601193,75.922820,$-543.187402955527,3.802395,7.407996,71.022413,0.070968,male,4.332644,-0.375737,1077.233497,13.177479,4.174536,NaN,no,2.605279,Texas,56.674425,0.108486,69.798923,30.684074,31.049447,5-10 miles,14.372443,100087.339539,0.869508,0.150728,NaN,-0.856591,-2.561083,50.236892,63.975108,6.998154,3756.910196,1.772648,15.057485,59.428690,1.844493,NaN,4.127857,NaN,182.369349,0,September,6.228138,1.370661,-0.239705,54.120933,geico,-0.032980,14.402071,18.211817,-0.819798,1.010811,-0.000234,48.202036,9.336021,0.209817,1.124866,-3.558718,subaru,55.020333,1.0,5.660882,-2.608974,-6.387984,2.506272,2.138771,NaN,53.977291,0.657292,-0.353469,NaN,3.997183,2.059615,0.305170,no,4.456565,NaN,8.754572,7.810979,0,yes,109.345215,Thursday,0,0,0,0,0,0,1,0,0,0,0,0,1,0
3,0.0,3.925235,19.901257,Tuesday,1.763602,-0.251926,-0.827461,-0.0057%,-0.520756,1.825586,2.223038,96.420382,-1.390239,3.962961,NaN,-2.046856,NaN,6.611554,74.966925,$-182.626380634258,7.728963,8.136213,121.610846,0.700954,male,7.294990,-0.603983,1051.655489,17.006528,2.347355,NaN,no,1.071202,Minnesota,59.154933,1.319711,65.408246,34.401290,48.363690,5-10 miles,13.191173,100934.096543,NaN,-0.965711,NaN,0.422522,-2.123048,41.857197,59.226119,NaN,1961.609788,3.155214,NaN,68.671023,-1.020225,5.833712,0.663759,NaN,300.629990,0,September,6.005140,0.013162,0.318335,54.784192,geico,-0.466535,14.402071,14.629914,1.389325,0.704880,-1.510949,49.882647,5.661421,1.606797,1.726010,-0.398417,nissan,47.769343,1.0,7.472328,1.424316,-5.431998,3.285291,2.138771,105.208424,49.543472,2.066346,1.761266,NaN,2.933707,0.899392,5.971782,no,4.100022,1.151085,NaN,9.178325,1,yes,103.021970,Tuesday,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,0.0,2.868802,22.202473,Sunday,3.405119,0.083162,1.381504,0.0109%,-0.732739,2.151990,-0.275406,90.769952,7.230125,3.877312,0.392002,-1.201565,100.626558,9.103015,77.977156,$967.007090837503,2.272765,24.452102,56.492317,-1.102387,male,6.313827,0.429187,949.904947,16.962710,0.510985,NaN,yes,2.283921,New York,46.445617,0.022747,66.662910,31.135261,31.819899,5-10 miles,17.210430,NaN,NaN,1.012093,NaN,-0.348240,3.477451,47.844153,55.921988,NaN,2345.195505,3.253079,14.193721,71.853326,0.926779,4.115990,-2.273628,NaN,149.725023,0,January,7.654926,1.305936,0.418272,51.486405,geico,-0.195764,14.402071,12.227512,-2.951163,1.096548,-0.000234,51.349106,9.422401,3.488398,1.884259,0.019803,toyota,44.640410,1.0,6.530625,0.705003,-5.664815,3.395916,2.138771,96.150945,54.843346,0.663113,-0.952377,NaN,2.922302,3.003595,1.046096,yes,3.234033,2.074927,9.987006,11.702664,0,yes,92.925935,Sunday,0,1,0,0,0,1,0,0,0,0,0,0,1,0
13,0.0,3.671553,22.207880,Tue,0.698724,-1.862047,2.153832,0.0068%,0.205453,1.309530,0.683070,111.472466,1.433037,2.625811,0.313683,-0.824623,102.816034,6.134170,72.430917,$-377.495529325022,7.074143,7.676579,72.703358,0.520186,male,0.849677,1.178534,759.765566,17.523317,0.444232,-1.549220,no,1.130353,Florida,41.982861,2.309857,68.489433,31.802446,29.893769,5-10 miles,16.987226,97628.442038,NaN,0.440045,NaN,0.672879,3.121838,50.335336,55.979128,9.887514,1564.639941,3.842460,NaN,73.728585,1.366928,NaN,0.186035,47.142610,300.629990,0,January,

In [136]:
catStateX33 = pd.get_dummies(dfFilter["x33"],prefix="cat_state_x33")
dfFilter = pd.merge(
    left=dfFilter,
    right=catStateX33,
    left_index=True,
    right_index=True,
)

dfFilter.head(15)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,map_days_x3,encode_gender_x24,encode_x31,cat_days_x3_Friday,cat_days_x3_Monday,cat_days_x3_Saturday,cat_days_x3_Sunday,cat_days_x3_Thursday,cat_days_x3_Tuesday,cat_days_x3_Wednesday,cat_insurance_x65_allstate,cat_insurance_x65_esurance,cat_insurance_x65_farmers,cat_insurance_x65_geico,cat_insurance_x65_progressive,cat_state_x33_Alabama,cat_state_x33_Alaska,cat_state_x33_Arizona,cat_state_x33_Arkansas,cat_state_x33_California,cat_state_x33_Colorado,cat_state_x33_Connecticut,cat_state_x33_DC,cat_state_x33_Delaware,cat_state_x33_Florida,cat_state_x33_Georgia,cat_state_x33_Hawaii,cat_state_x33_Idaho,cat_state_x33_Illinois,cat_state_x33_Indiana,cat_state_x33_Iowa,cat_state_x33_Kansas,cat_state_x33_Kentucky,cat_state_x33_Louisiana,cat_state_x33_Maine,cat_state_x33_Maryland,cat_state_x33_Massachusetts,cat_state_x33_Michigan,cat_state_x33_Minnesota,cat_state_x33_Mississippi,cat_state_x33_Missouri,cat_state_x33_Montana,cat_state_x33_Nebraska,cat_state_x33_Nevada,cat_state_x33_New Hampshire,cat_state_x33_New Jersey,cat_state_x33_New Mexico,cat_state_x33_New York,cat_state_x33_North Carolina,cat_state_x33_North Dakota,cat_state_x33_Ohio,cat_state_x33_Oklahoma,cat_state_x33_Oregon,cat_state_x33_Pennsylvania,cat_state_x33_Rhode Island,cat_state_x33_South Carolina,cat_state_x33_South Dakota,cat_state_x33_Tennessee,cat_state_x33_Texas,cat_state_x33_Utah,cat_state_x33_Vermont,cat_state_x33_Virginia,cat_state_x33_Washington,cat_state_x33_West Virginia,cat_state_x33_Wisconsin,cat_state_x33_Wyoming
2,1.0,4.427278,19.188092,Thursday,0.145652,0.366093,0.709962,-8e-04%,0.952323,0.782974,-1.247022,95.375221,1.098525,1.216059,0.450624,0.211685,99.215210,8.601193,75.922820,$-543.187402955527,3.802395,7.407996,71.022413,0.070968,male,4.332644,-0.375737,1077.233497,13.177479,4.174536,NaN,no,2.605279,Texas,56.674425,0.108486,69.798923,30.684074,31.049447,5-10 miles,14.372443,100087.339539,0.869508,0.150728,NaN,-0.856591,-2.561083,50.236892,63.975108,6.998154,3756.910196,1.772648,15.057485,59.428690,1.844493,NaN,4.127857,NaN,182.369349,0,September,6.228138,1.370661,-0.239705,54.120933,geico,-0.032980,14.402071,18.211817,-0.819798,1.010811,-0.000234,48.202036,9.336021,0.209817,1.124866,-3.558718,subaru,55.020333,1.0,5.660882,-2.608974,-6.387984,2.506272,2.138771,NaN,53.977291,0.657292,-0.353469,NaN,3.997183,2.059615,0.305170,no,4.456565,NaN,8.754572,7.810979,0,yes,109.345215,Thursday,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,0.0,3.925235,19.901257,Tuesday,1.763602,-0.251926,-0.827461,-0.0057%,-0.520756,1.825586,2.223038,96.420382,-1.390239,3.962961,NaN,-2.046856,NaN,6.611554,74.966925,$-182.626380634258,7.728963,8.136213,121.610846,0.700954,male,7.294990,-0.603983,1051.655489,17.006528,2.347355,NaN,no,1.071202,Minnesota,59.154933,1.319711,65.408246,34.401290,48.363690,5-10 miles,13.191173,100934.096543,NaN,-0.965711,NaN,0.422522,-2.123048,41.857197,59.226119,NaN,1961.609788,3.155214,NaN,68.671023,-1.020225,5.833712,0.663759,NaN,300.629990,0,September,6.005140,0.013162,0.318335,54.784192,geico,-0.466535,14.402071,14.629914,1.389325,0.704880,-1.510949,49.882647,5.661421,1.606797,1.726010,-0.398417,nissan,47.769343,1.0,7.472328,1.424316,-5.431998,3.285291,2.138771,105.208424,49.543472,2.066346,1.761266,NaN,2.933707,0.899392,5.971782,no,4.100022,1.151085,NaN,9.178325,1,yes,103.021970,Tuesday,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0.0,2.868802,22.202473,Sunday,3.405119,0.083162,1.381504,0.0109%

In [137]:
catCarManufacturerX77 = pd.get_dummies(dfFilter["x77"],prefix="cat_car_manufacturer_x77")
dfFilter = pd.merge(
    left=dfFilter,
    right=catCarManufacturerX77,
    left_index=True,
    right_index=True,
)

dfFilter.head(15)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,map_days_x3,encode_gender_x24,encode_x31,cat_days_x3_Friday,cat_days_x3_Monday,cat_days_x3_Saturday,cat_days_x3_Sunday,cat_days_x3_Thursday,cat_days_x3_Tuesday,cat_days_x3_Wednesday,cat_insurance_x65_allstate,cat_insurance_x65_esurance,cat_insurance_x65_farmers,cat_insurance_x65_geico,cat_insurance_x65_progressive,cat_state_x33_Alabama,cat_state_x33_Alaska,cat_state_x33_Arizona,cat_state_x33_Arkansas,cat_state_x33_California,cat_state_x33_Colorado,cat_state_x33_Connecticut,cat_state_x33_DC,cat_state_x33_Delaware,cat_state_x33_Florida,cat_state_x33_Georgia,cat_state_x33_Hawaii,cat_state_x33_Idaho,cat_state_x33_Illinois,cat_state_x33_Indiana,cat_state_x33_Iowa,cat_state_x33_Kansas,cat_state_x33_Kentucky,cat_state_x33_Louisiana,cat_state_x33_Maine,cat_state_x33_Maryland,cat_state_x33_Massachusetts,cat_state_x33_Michigan,cat_state_x33_Minnesota,cat_state_x33_Mississippi,cat_state_x33_Missouri,cat_state_x33_Montana,cat_state_x33_Nebraska,cat_state_x33_Nevada,cat_state_x33_New Hampshire,cat_state_x33_New Jersey,cat_state_x33_New Mexico,cat_state_x33_New York,cat_state_x33_North Carolina,cat_state_x33_North Dakota,cat_state_x33_Ohio,cat_state_x33_Oklahoma,cat_state_x33_Oregon,cat_state_x33_Pennsylvania,cat_state_x33_Rhode Island,cat_state_x33_South Carolina,cat_state_x33_South Dakota,cat_state_x33_Tennessee,cat_state_x33_Texas,cat_state_x33_Utah,cat_state_x33_Vermont,cat_state_x33_Virginia,cat_state_x33_Washington,cat_state_x33_West Virginia,cat_state_x33_Wisconsin,cat_state_x33_Wyoming,cat_car_manufacturer_x77_buick,cat_car_manufacturer_x77_chevrolet,cat_car_manufacturer_x77_ford,cat_car_manufacturer_x77_mercedes,cat_car_manufacturer_x77_nissan,cat_car_manufacturer_x77_subaru,cat_car_manufacturer_x77_toyota
2,1.0,4.427278,19.188092,Thursday,0.145652,0.366093,0.709962,-8e-04%,0.952323,0.782974,-1.247022,95.375221,1.098525,1.216059,0.450624,0.211685,99.215210,8.601193,75.922820,$-543.187402955527,3.802395,7.407996,71.022413,0.070968,male,4.332644,-0.375737,1077.233497,13.177479,4.174536,NaN,no,2.605279,Texas,56.674425,0.108486,69.798923,30.684074,31.049447,5-10 miles,14.372443,100087.339539,0.869508,0.150728,NaN,-0.856591,-2.561083,50.236892,63.975108,6.998154,3756.910196,1.772648,15.057485,59.428690,1.844493,NaN,4.127857,NaN,182.369349,0,September,6.228138,1.370661,-0.239705,54.120933,geico,-0.032980,14.402071,18.211817,-0.819798,1.010811,-0.000234,48.202036,9.336021,0.209817,1.124866,-3.558718,subaru,55.020333,1.0,5.660882,-2.608974,-6.387984,2.506272,2.138771,NaN,53.977291,0.657292,-0.353469,NaN,3.997183,2.059615,0.305170,no,4.456565,NaN,8.754572,7.810979,0,yes,109.345215,Thursday,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0.0,3.925235,19.901257,Tuesday,1.763602,-0.251926,-0.827461,-0.0057%,-0.520756,1.825586,2.223038,96.420382,-1.390239,3.962961,NaN,-2.046856,NaN,6.611554,74.966925,$-182.626380634258,7.728963,8.136213,121.610846,0.700954,male,7.294990,-0.603983,1051.655489,17.006528,2.347355,NaN,no,1.071202,Minnesota,59.154933,1.319711,65.408246,34.401290,48.363690,5-10 miles,13.191173,100934.096543,NaN,-0.965711,NaN,0.422522,-2.123048,41.857197,59.226119,NaN,1961.609788,3.155214,NaN,68.671023,-1.020225,5.833712,0.663759,NaN,300.629990,0,September,6.005140,0.013162,0.318335,54.784192,geico,-0.466535,14.402071,14.629914,1.389325,0.704880,-1.510949,49.882647,5.661421,1.606797,1.726010,-0.398417,nissan,47.769343,1.0,7.472328,1.424316,-5.431998,3.285291,2.138771,105.208424,49.543472,2.066346,1.761266,NaN,2.933707,0.899392,5.971782,no,4.100022,1.1

In [138]:
dfFilter.columns.tolist()

['y',
 'x1',
 'x2',
 'x3',
 'x4',
 'x5',
 'x6',
 'x7',
 'x8',
 'x9',
 'x10',
 'x11',
 'x12',
 'x13',
 'x14',
 'x15',
 'x16',
 'x17',
 'x18',
 'x19',
 'x20',
 'x21',
 'x22',
 'x23',
 'x24',
 'x25',
 'x26',
 'x27',
 'x28',
 'x29',
 'x30',
 'x31',
 'x32',
 'x33',
 'x34',
 'x35',
 'x36',
 'x37',
 'x38',
 'x39',
 'x40',
 'x41',
 'x42',
 'x43',
 'x44',
 'x45',
 'x46',
 'x47',
 'x48',
 'x49',
 'x50',
 'x51',
 'x52',
 'x53',
 'x54',
 'x55',
 'x56',
 'x57',
 'x58',
 'x59',
 'x60',
 'x61',
 'x62',
 'x63',
 'x64',
 'x65',
 'x66',
 'x67',
 'x68',
 'x69',
 'x70',
 'x71',
 'x72',
 'x73',
 'x74',
 'x75',
 'x76',
 'x77',
 'x78',
 'x79',
 'x80',
 'x81',
 'x82',
 'x83',
 'x84',
 'x85',
 'x86',
 'x87',
 'x88',
 'x89',
 'x90',
 'x91',
 'x92',
 'x93',
 'x94',
 'x95',
 'x96',
 'x97',
 'x98',
 'x99',
 'x100',
 'map_days_x3',
 'encode_gender_x24',
 'encode_x31',
 'cat_days_x3_Friday',
 'cat_days_x3_Monday',
 'cat_days_x3_Saturday',
 'cat_days_x3_Sunday',
 'cat_days_x3_Thursday',
 'cat_days_x3_Tuesday',
 'cat_

## To be continued...

### Interpolate missing values

In [9]:
dfCombine.interpolate(method="linear",inplace=True)
dfCombine.sample(25)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x12,x13,x15,x17,x18,x19,x20,x21,x22,x23,x25,x26,x27,x28,x29,x31,x32,x34,x35,x36,x37,x40,x43,x46,x47,x48,x50,x51,x53,x56,x58,x59,x60,x62,x63,x65,x66,x67,x68,x69,x70,x71,x72,x73,x81,x82,x84,x87,x90,x92,x93,x94,x97,x98,x100
5939,0.0,5.502680,17.780351,Monday,0.380849,1.004747,-0.438663,0.0078%,0.739941,1.828471,1.383963,-2.037298,3.013783,-0.095031,6.883429,82.127349,$-1710.85250393773,8.632427,-0.674212,69.536503,-0.151843,4.382201,1.011935,938.339657,15.297491,2.617807,no,0.777178,56.954778,1.081001,71.416683,32.681625,15.679302,0.092683,1.262303,48.444046,54.986591,2603.799962,1.612120,68.609594,3.554496,300.629990,0,January,4.853909,-0.336753,progressive,0.346917,14.402071,16.003546,3.166527,0.560094,-0.465103,43.970585,3.603619,-1.404248,-7.909875,2.138771,0.112778,2.785838,6.950069,no,4.274283,9.946776,0,100.539412
30366,0.0,2.282462,21.162563,Tue,-1.808235,0.661000,-0.625814,-9e-04%,1.483337,0.959319,-1.930960,2.942658,2.140495,2.964864,5.975118,86.502088,$-660.359028996502,7.353357,1.899780,62.150731,0.868181,2.003517,-0.162476,1041.822215,13.760070,0.311323,no,0.037321,56.117605,-1.320441,73.454275,29.764676,12.277456,0.666767,0.443973,51.091937,62.696822,1825.291294,2.447103,74.112632,2.298110,300.629990,0,January,3.663045,1.560442,esurance,0.255694,14.402071,19.638469,1.038815,0.532548,-0.000234,42.401437,5.822040,-1.721292,-7.107652,2.138771,0.187328,1.513155,3.940189,no,3.919825,7.095649,1,105.960097
28782,1.0,-0.344371,20.339917,Friday,1.038342,-1.607311,-1.702069,-0.0117%,-1.018598,5.616029,0.205000,8.400455,2.371640,-0.980639,8.448383,81.711906,$-1399.95598337753,3.036741,7.641825,48.429785,-0.329066,4.280220,0.635909,898.083665,20.055415,0.576180,no,-0.478091,42.535660,2.387067,64.137974,34.155697,17.679606,0.196375,-0.637558,45.137640,60.865660,1974.777352,1.345923,82.238939,-0.200511,300.629990,0,December,4.631211,2.988295,progressive,-0.849834,14.402071,17.881365,0.036205,0.228168,-0.000234,53.244575,8.551170,-3.578433,-4.267055,2.138771,-0.420897,3.590883,4.768065,no,4.915783,11.987642,0,101.130022
35686,0.0,2.026539,20.753612,Tue,-1.934462,-0.793248,0.343086,-0.0033%,0.362065,1.627749,1.465244,3.345612,0.707100,0.219488,6.310715,76.177374,$273.747478576495,5.296041,19.692126,56.813788,-0.899661,8.523826,-1.323164,994.604961,12.439035,0.555761,no,-0.497708,57.612911,0.645619,70.207192,28.468189,14.142455,-0.486875,1.263802,52.794273,55.808994,1412.878661,2.198346,71.847701,2.251160,300.629990,0,December,0.910599,-1.341943,geico,0.699843,14.402071,17.359504,1.130938,0.756249,-2.802459,46.034234,-0.007728,2.085593,-9.418618,2.138771,-1.029165,1.694681,2.166077,yes,1.897776,11.680951,1,98.805197
19256,1.0,4.469518,21.926628,Tuesday,-0.159141,1.441846,1.008905,-6e-04%,-1.142222,0.519749,0.574060,-1.702029,2.438225,1.352324,10.812144,69.968075,$-626.171961919776,6.058290,38.997200,31.603428,-1.199585,6.508440,-1.245456,1087.601639,21.648042,2.332966,no,1.105912,60.019274,-0.918577,62.978380,27.053928,11.902635,2.065798,2.523332,45.383477,60.803596,1395.298378,-0.108095,74.340676,0.568177,300.629990,0,December,0.408142,1.217035,progressive,-0.079943,14.402071,21.884022,-2.349334,1.082790,-0.000234,51.975065,10.678767,1.707492,-9.370670,2.138771,-0.628366,3.556205,5.851769,no,4.111224,11.994903,1,92.427980
12364,0.0,2.729361,16.322957,Saturday,1.705681,0.270789,-2.360248,-0.0195%,0.927919,2.936379,1.267098,5.529965,3.956167,-0.536040,7.311113,76.202662,$139.798572989896,8.470573,4.198530,51.344545,-1.077694,1.140622,-0.338667,996.372230,16.246598,-0.674901,no,0.049368,55.471581,1.759944,67.241502,36.089306,19.670711,0.044632,-1.487133,49.793350,56.521135,1672.905232,0.436104,72.655048,-1.060696,300.629990,0,July,4.814890,0.072426,allstate,-0.858030,14.402071,13.983871,0.172833,0.383715,-0.000234,40.115952,7.798735,1.158259,-5.203260,2.138771,0.560397,4.794093,7.090674,no,6.443068,8.711088,1,107.837004
24558,0.0,2.036445,21.017509,Sunday,-2.788471,1.054235,1.396404,0.0175%,-2.049304,

### x7
Remove percent sign

In [14]:
dfCombine['x7'] = dfCombine["x7"].str.replace("%","")

In [15]:
dfCombine.head(5)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x12,x13,x15,x17,x18,x19,x20,x21,x22,x23,x25,x26,x27,x28,x29,x31,x32,x34,x35,x36,x37,x40,x43,x46,x47,x48,x50,x51,x53,x56,x58,x59,x60,x62,x63,x65,x66,x67,x68,x69,x70,x71,x72,x73,x81,x82,x84,x87,x90,x92,x93,x94,x97,x98,x100
0,0.0,0.165254,18.060003,Wednesday,1.077380,-1.339233,-1.584341,0.0062,0.220784,1.816481,1.171788,4.644568,4.814885,-0.587361,4.278444,77.709700,$-908.650758424405,7.328029,36.601967,126.384294,1.264713,3.834082,2.352406,905.491445,14.779467,0.752524,no,-0.597288,46.667221,3.159002,69.444841,32.450076,18.129499,1.227266,0.234654,53.359113,58.251700,1602.756464,2.740134,72.167186,-1.532562,300.629990,0,August,6.600008,1.252479,farmers,1.212048,12.896733,15.263491,0.518653,0.543768,-0.912506,53.521544,7.723719,-0.471594,-3.261865,0.383351,-0.162668,2.770844,0.909479,no,5.492487,7.627730,0,104.251338
1,1.0,2.441471,18.416307,Friday,1.482586,0.920817,-0.759931,0.0064,1.192441,3.513950,1.419900,1.459868,1.443983,-1.165869,8.487132,70.808566,$-1864.9622875143,6.796408,3.789176,93.037021,0.346201,4.345028,1.651579,868.784447,25.914680,1.445294,no,1.127376,47.928569,-1.013712,61.289132,35.521824,11.389081,-0.822055,1.104631,37.728834,56.052749,3350.837875,1.995635,73.964560,0.956001,300.629990,0,April,3.864254,0.057936,allstate,0.738526,14.402071,33.940951,-0.140384,1.016703,-0.000234,46.797571,7.260365,1.691428,-4.789183,2.138771,1.252022,3.703013,9.093466,no,3.346429,10.505284,1,101.230645
2,1.0,4.427278,19.188092,Thursday,0.145652,0.366093,0.709962,-8e-04,0.952323,0.782974,-1.247022,1.098525,1.216059,0.211685,8.601193,75.922820,$-543.187402955527,3.802395,7.407996,71.022413,0.070968,4.332644,-0.375737,1077.233497,13.177479,4.174536,no,2.605279,56.674425,0.108486,69.798923,30.684074,14.372443,0.150728,-2.561083,50.236892,63.975108,3756.910196,1.772648,59.428690,4.127857,182.369349,0,September,1.370661,-0.239705,geico,-0.032980,14.402071,18.211817,-0.819798,1.010811,-0.000234,48.202036,9.336021,-2.608974,-6.387984,2.138771,0.657292,3.997183,0.305170,no,4.456565,7.810979,0,109.345215
3,0.0,3.925235,19.901257,Tuesday,1.763602,-0.251926,-0.827461,-0.0057,-0.520756,1.825586,2.223038,-1.390239,3.962961,-2.046856,6.611554,74.966925,$-182.626380634258,7.728963,8.136213,121.610846,0.700954,7.294990,-0.603983,1051.655489,17.006528,2.347355,no,1.071202,59.154933,1.319711,65.408246,34.401290,13.191173,-0.965711,-2.123048,41.857197,59.226119,1961.609788,3.155214,68.671023,0.663759,300.629990,0,September,0.013162,0.318335,geico,-0.466535,14.402071,14.629914,1.389325,0.704880,-1.510949,49.882647,5.661421,1.424316,-5.431998,2.138771,2.066346,2.933707,5.971782,no,4.100022,9.178325,1,103.021970
4,0.0,2.868802,22.202473,Sunday,3.405119,0.083162,1.381504,0.0109,-0.732739,2.151990,-0.275406,7.230125,3.877312,-1.201565,9.103015,77.977156,$967.007090837503,2.272765,24.452102,56.492317,-1.102387,6.313827,0.429187,949.904947,16.962710,0.510985,yes,2.283921,46.445617,0.022747,66.662910,31.135261,17.210430,1.012093,3.477451,47.844153,55.921988,2345.195505,3.253079,71.853326,-2.273628,149.725023,0,January,1.305936,0.418272,geico,-0.195764,14.402071,12.227512,-2.951163,1.096548,-0.000234,51.349106,9.422401,0.705003,-5.664815,2.138771,0.663113,2.922302,1.046096,yes,3.234033,11.702664,0,92.925935


In [16]:
# remove "E" notation
dfCombine["x7"] = pd.to_numeric(dfCombine["x7"])
dfCombine.sample(25)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x12,x13,x15,x17,x18,x19,x20,x21,x22,x23,x25,x26,x27,x28,x29,x31,x32,x34,x35,x36,x37,x40,x43,x46,x47,x48,x50,x51,x53,x56,x58,x59,x60,x62,x63,x65,x66,x67,x68,x69,x70,x71,x72,x73,x81,x82,x84,x87,x90,x92,x93,x94,x97,x98,x100
19705,0.0,6.004367,20.570062,Wednesday,-1.457530,-1.607966,-0.471952,-0.0168,-0.144604,4.297616,0.804386,7.600365,3.658712,-0.255275,7.651588,73.050068,$1595.35915502259,1.276173,-0.260140,89.718116,-0.347197,5.505008,-2.580260,956.991452,17.346864,-1.233363,no,-0.545354,43.909523,3.382840,65.336651,30.593144,19.588008,0.675316,-0.803257,54.700192,67.365398,270.088793,-0.081847,79.383264,2.335661,333.446329,0,December,3.966645,3.621353,progressive,-0.708436,17.313624,17.171236,1.683793,0.504219,-0.000234,62.427284,5.406381,0.456705,-4.893025,2.138771,1.138440,3.651768,1.468252,no,3.635328,11.944268,0,94.169854
17152,0.0,5.680569,15.729303,Tuesday,-0.065580,0.233062,-3.395908,0.0025,0.085624,7.270380,1.794059,0.976688,4.992474,1.080229,7.893710,88.053142,$-19.0102742027183,2.581499,8.599259,77.711926,-0.243012,7.215930,0.840597,1017.451690,10.634885,0.236074,yes,0.104853,57.042278,3.992951,72.147029,32.926793,19.464715,0.972463,-0.394235,59.462580,65.958632,-264.084713,0.751506,80.197130,-1.134119,300.629990,0,July,2.867396,-0.027181,esurance,-1.171031,14.402071,6.999596,0.512064,0.237246,-0.000234,40.839575,11.855895,1.357583,-5.639372,2.138771,1.026432,5.370136,8.305195,no,5.790063,6.096135,0,107.218797
21189,0.0,3.674369,19.666963,Monday,0.069759,0.012630,-2.452687,0.0102,0.835478,1.237607,2.191204,-2.050225,4.375355,-0.810691,4.776027,84.451131,$2270.98437408929,10.864673,9.375025,59.265743,1.307909,7.234530,-0.227096,866.464126,15.330462,0.166689,no,-1.889282,47.826455,0.317373,67.389964,32.277445,15.985112,0.015715,0.258946,49.053371,54.353331,2811.339508,-0.724428,78.335425,1.815228,300.629990,0,January,2.604229,3.002418,progressive,0.657315,14.402071,16.774300,-0.288372,0.541533,-0.000234,58.359396,6.096790,-0.681295,-4.325262,2.138771,0.215095,4.856121,-1.550906,no,3.025752,9.163230,1,100.685268
43275,0.0,3.942453,20.541904,Thursday,-3.528383,-1.930628,-0.683623,0.0011,1.678000,2.386958,-0.163503,-0.735059,4.846069,0.286632,11.504146,89.378214,$1216.99221268236,4.414558,0.990106,91.475387,0.592597,7.616571,1.133611,1186.472615,11.266575,-0.581559,no,-0.522054,65.303920,0.832703,75.948442,27.574844,15.813808,1.101209,-1.464038,54.461682,68.266088,812.985864,0.587754,80.055324,0.723613,300.629990,0,December,3.713186,0.203542,allstate,0.110158,14.402071,6.622094,0.949597,0.215628,-0.000234,50.590482,5.556190,-0.046981,-6.973435,2.138771,-0.406946,1.556854,3.085950,no,2.207574,9.062181,1,99.165214
30,0.0,5.425648,18.757789,Thursday,0.741306,0.705725,-0.227966,-0.0118,1.056694,1.362535,-0.417212,4.750968,2.726950,0.659888,7.727316,76.248936,$-1258.81184572961,3.713467,1.975054,99.044943,0.106004,7.626973,-0.413595,866.066918,14.529850,1.850933,no,-1.360002,53.645042,2.798853,68.282641,30.293543,18.173743,1.692109,2.541289,52.646384,55.868266,3567.558053,3.024054,69.220579,1.610264,300.629990,0,July,7.797542,0.218372,allstate,-0.590423,14.402071,15.548843,0.935956,0.658770,-0.000234,49.065051,4.257907,-2.132114,-5.879097,2.138771,-0.227429,4.345416,8.182267,no,3.420189,9.318231,1,94.051198
3542,0.0,7.978522,20.073679,Saturday,1.142898,3.036605,2.013422,0.0136,-2.031602,2.779486,0.668535,-0.773224,9.604079,-0.117346,11.882598,80.461755,$-149.672141831441,10.118640,75.276994,37.685289,0.015340,6.583487,-1.643494,974.799051,19.428675,-0.224320,no,2.009091,62.086159,0.642766,72.296180,32.293048,10.048753,0.516852,-0.013440,44.001141,62.371872,1208.446097,1.174323,64.769621,-0.829404,487.854878,0,January,5.627259,-3.298944,farmers,-1.753108,14.402071,13.546280,-1.167964,0.781905,1.652105,46.132460,21.984539,1.610197,-6.909327,2.138771,-0.429800,1.802602,1.664215,no,2.737327,8.861015,1,93.836818
36976,0.0,2.493125,20.347179,Wednesday,-0.340143,1.963206,0.681293,0.0045,1.985813,4.1

### x19

In [17]:
# replace dollar sign
dfCombine["x19"] = dfCombine["x19"].replace(to_replace="[\$]",value="",regex=True)
dfCombine.head(10)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x12,x13,x15,x17,x18,x19,x20,x21,x22,x23,x25,x26,x27,x28,x29,x31,x32,x34,x35,x36,x37,x40,x43,x46,x47,x48,x50,x51,x53,x56,x58,x59,x60,x62,x63,x65,x66,x67,x68,x69,x70,x71,x72,x73,x81,x82,x84,x87,x90,x92,x93,x94,x97,x98,x100
0,0.0,0.165254,18.060003,Wednesday,1.077380,-1.339233,-1.584341,0.0062,0.220784,1.816481,1.171788,4.644568,4.814885,-0.587361,4.278444,77.709700,-908.650758424405,7.328029,36.601967,126.384294,1.264713,3.834082,2.352406,905.491445,14.779467,0.752524,no,-0.597288,46.667221,3.159002,69.444841,32.450076,18.129499,1.227266,0.234654,53.359113,58.251700,1602.756464,2.740134,72.167186,-1.532562,300.629990,0,August,6.600008,1.252479,farmers,1.212048,12.896733,15.263491,0.518653,0.543768,-0.912506,53.521544,7.723719,-0.471594,-3.261865,0.383351,-0.162668,2.770844,0.909479,no,5.492487,7.627730,0,104.251338
1,1.0,2.441471,18.416307,Friday,1.482586,0.920817,-0.759931,0.0064,1.192441,3.513950,1.419900,1.459868,1.443983,-1.165869,8.487132,70.808566,-1864.9622875143,6.796408,3.789176,93.037021,0.346201,4.345028,1.651579,868.784447,25.914680,1.445294,no,1.127376,47.928569,-1.013712,61.289132,35.521824,11.389081,-0.822055,1.104631,37.728834,56.052749,3350.837875,1.995635,73.964560,0.956001,300.629990,0,April,3.864254,0.057936,allstate,0.738526,14.402071,33.940951,-0.140384,1.016703,-0.000234,46.797571,7.260365,1.691428,-4.789183,2.138771,1.252022,3.703013,9.093466,no,3.346429,10.505284,1,101.230645
2,1.0,4.427278,19.188092,Thursday,0.145652,0.366093,0.709962,-0.0008,0.952323,0.782974,-1.247022,1.098525,1.216059,0.211685,8.601193,75.922820,-543.187402955527,3.802395,7.407996,71.022413,0.070968,4.332644,-0.375737,1077.233497,13.177479,4.174536,no,2.605279,56.674425,0.108486,69.798923,30.684074,14.372443,0.150728,-2.561083,50.236892,63.975108,3756.910196,1.772648,59.428690,4.127857,182.369349,0,September,1.370661,-0.239705,geico,-0.032980,14.402071,18.211817,-0.819798,1.010811,-0.000234,48.202036,9.336021,-2.608974,-6.387984,2.138771,0.657292,3.997183,0.305170,no,4.456565,7.810979,0,109.345215
3,0.0,3.925235,19.901257,Tuesday,1.763602,-0.251926,-0.827461,-0.0057,-0.520756,1.825586,2.223038,-1.390239,3.962961,-2.046856,6.611554,74.966925,-182.626380634258,7.728963,8.136213,121.610846,0.700954,7.294990,-0.603983,1051.655489,17.006528,2.347355,no,1.071202,59.154933,1.319711,65.408246,34.401290,13.191173,-0.965711,-2.123048,41.857197,59.226119,1961.609788,3.155214,68.671023,0.663759,300.629990,0,September,0.013162,0.318335,geico,-0.466535,14.402071,14.629914,1.389325,0.704880,-1.510949,49.882647,5.661421,1.424316,-5.431998,2.138771,2.066346,2.933707,5.971782,no,4.100022,9.178325,1,103.021970
4,0.0,2.868802,22.202473,Sunday,3.405119,0.083162,1.381504,0.0109,-0.732739,2.151990,-0.275406,7.230125,3.877312,-1.201565,9.103015,77.977156,967.007090837503,2.272765,24.452102,56.492317,-1.102387,6.313827,0.429187,949.904947,16.962710,0.510985,yes,2.283921,46.445617,0.022747,66.662910,31.135261,17.210430,1.012093,3.477451,47.844153,55.921988,2345.195505,3.253079,71.853326,-2.273628,149.725023,0,January,1.305936,0.418272,geico,-0.195764,14.402071,12.227512,-2.951163,1.096548,-0.000234,51.349106,9.422401,0.705003,-5.664815,2.138771,0.663113,2.922302,1.046096,yes,3.234033,11.702664,0,92.925935
5,0.0,1.926053,19.524376,Saturday,3.702574,2.300792,-0.331106,0.0079,3.085443,0.472701,1.943171,-3.582885,3.522031,-2.799700,7.283226,87.734440,271.163475698959,15.770180,44.874132,45.729121,0.320526,2.245217,4.303686,1187.333248,8.139008,2.233107,no,0.488196,45.559596,0.359074,76.835865,37.602918,12.628248,1.452704,-1.573853,52.359238,46.900080,2959.298318,1.670786,67.591932,0.493780,300.629990,0,August,4.106257,-2.246804,progressive,-0.065794,14.402071,23.446751,-2.378431,1.393887,-0.000234,49.858508,18.739751,1.887206,-3.892529,2.138771,1.027475,1.682037,5.253010,no,4.183036,5.632881,0,97.129172
6,0.0,0.094162,18.894755,Thursday,2.027918,1.580408,1.190932,0.0174,-0.331730,2.178622,-0.454172,1.762009,2.125462,-0.907832,8.6

#### Round to 6 decimal places

In [18]:
dfCombine["x19"] = dfCombine["x19"].astype(float).round(6)
dfCombine.head(10)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x12,x13,x15,x17,x18,x19,x20,x21,x22,x23,x25,x26,x27,x28,x29,x31,x32,x34,x35,x36,x37,x40,x43,x46,x47,x48,x50,x51,x53,x56,x58,x59,x60,x62,x63,x65,x66,x67,x68,x69,x70,x71,x72,x73,x81,x82,x84,x87,x90,x92,x93,x94,x97,x98,x100
0,0.0,0.165254,18.060003,Wednesday,1.077380,-1.339233,-1.584341,0.0062,0.220784,1.816481,1.171788,4.644568,4.814885,-0.587361,4.278444,77.709700,-908.650758,7.328029,36.601967,126.384294,1.264713,3.834082,2.352406,905.491445,14.779467,0.752524,no,-0.597288,46.667221,3.159002,69.444841,32.450076,18.129499,1.227266,0.234654,53.359113,58.251700,1602.756464,2.740134,72.167186,-1.532562,300.629990,0,August,6.600008,1.252479,farmers,1.212048,12.896733,15.263491,0.518653,0.543768,-0.912506,53.521544,7.723719,-0.471594,-3.261865,0.383351,-0.162668,2.770844,0.909479,no,5.492487,7.627730,0,104.251338
1,1.0,2.441471,18.416307,Friday,1.482586,0.920817,-0.759931,0.0064,1.192441,3.513950,1.419900,1.459868,1.443983,-1.165869,8.487132,70.808566,-1864.962288,6.796408,3.789176,93.037021,0.346201,4.345028,1.651579,868.784447,25.914680,1.445294,no,1.127376,47.928569,-1.013712,61.289132,35.521824,11.389081,-0.822055,1.104631,37.728834,56.052749,3350.837875,1.995635,73.964560,0.956001,300.629990,0,April,3.864254,0.057936,allstate,0.738526,14.402071,33.940951,-0.140384,1.016703,-0.000234,46.797571,7.260365,1.691428,-4.789183,2.138771,1.252022,3.703013,9.093466,no,3.346429,10.505284,1,101.230645
2,1.0,4.427278,19.188092,Thursday,0.145652,0.366093,0.709962,-0.0008,0.952323,0.782974,-1.247022,1.098525,1.216059,0.211685,8.601193,75.922820,-543.187403,3.802395,7.407996,71.022413,0.070968,4.332644,-0.375737,1077.233497,13.177479,4.174536,no,2.605279,56.674425,0.108486,69.798923,30.684074,14.372443,0.150728,-2.561083,50.236892,63.975108,3756.910196,1.772648,59.428690,4.127857,182.369349,0,September,1.370661,-0.239705,geico,-0.032980,14.402071,18.211817,-0.819798,1.010811,-0.000234,48.202036,9.336021,-2.608974,-6.387984,2.138771,0.657292,3.997183,0.305170,no,4.456565,7.810979,0,109.345215
3,0.0,3.925235,19.901257,Tuesday,1.763602,-0.251926,-0.827461,-0.0057,-0.520756,1.825586,2.223038,-1.390239,3.962961,-2.046856,6.611554,74.966925,-182.626381,7.728963,8.136213,121.610846,0.700954,7.294990,-0.603983,1051.655489,17.006528,2.347355,no,1.071202,59.154933,1.319711,65.408246,34.401290,13.191173,-0.965711,-2.123048,41.857197,59.226119,1961.609788,3.155214,68.671023,0.663759,300.629990,0,September,0.013162,0.318335,geico,-0.466535,14.402071,14.629914,1.389325,0.704880,-1.510949,49.882647,5.661421,1.424316,-5.431998,2.138771,2.066346,2.933707,5.971782,no,4.100022,9.178325,1,103.021970
4,0.0,2.868802,22.202473,Sunday,3.405119,0.083162,1.381504,0.0109,-0.732739,2.151990,-0.275406,7.230125,3.877312,-1.201565,9.103015,77.977156,967.007091,2.272765,24.452102,56.492317,-1.102387,6.313827,0.429187,949.904947,16.962710,0.510985,yes,2.283921,46.445617,0.022747,66.662910,31.135261,17.210430,1.012093,3.477451,47.844153,55.921988,2345.195505,3.253079,71.853326,-2.273628,149.725023,0,January,1.305936,0.418272,geico,-0.195764,14.402071,12.227512,-2.951163,1.096548,-0.000234,51.349106,9.422401,0.705003,-5.664815,2.138771,0.663113,2.922302,1.046096,yes,3.234033,11.702664,0,92.925935
5,0.0,1.926053,19.524376,Saturday,3.702574,2.300792,-0.331106,0.0079,3.085443,0.472701,1.943171,-3.582885,3.522031,-2.799700,7.283226,87.734440,271.163476,15.770180,44.874132,45.729121,0.320526,2.245217,4.303686,1187.333248,8.139008,2.233107,no,0.488196,45.559596,0.359074,76.835865,37.602918,12.628248,1.452704,-1.573853,52.359238,46.900080,2959.298318,1.670786,67.591932,0.493780,300.629990,0,August,4.106257,-2.246804,progressive,-0.065794,14.402071,23.446751,-2.378431,1.393887,-0.000234,49.858508,18.739751,1.887206,-3.892529,2.138771,1.027475,1.682037,5.253010,no,4.183036,5.632881,0,97.129172
6,0.0,0.094162,18.894755,Thursday,2.027918,1.580408,1.190932,0.0174,-0.331730,2.178622,-0.454172,1.762009,2.125462,-0.907832,8.631917,71.282134,749.965010,14.2208

### x93

In [20]:
dfCombine["x93"] = dfCombine["x93"].map({"no":0,"yes":1})
dfCombine["x93"].unique()

array([0, 1], dtype=int64)

### Logistic regression (classification) & neural network

class sklearn.linear_model.LogisticRegression(penalty='l2',dual=False,tol=0.0001,C=1.0,fit_intercept=True,
intercept_scaling=1,class_weight=None,random_state=None,solver='lbfgs',max_iter=100,multi_class='auto', 
verbose=0,warm_start=False,n_jobs=None,l1_ratio=None)

In [25]:
dfCombine.columns

Index(['y', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x12',
       'x13', 'x15', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23', 'x25',
       'x26', 'x27', 'x28', 'x29', 'x31', 'x32', 'x34', 'x35', 'x36', 'x37',
       'x40', 'x43', 'x46', 'x47', 'x48', 'x50', 'x51', 'x53', 'x56', 'x58',
       'x59', 'x60', 'x62', 'x63', 'x65', 'x66', 'x67', 'x68', 'x69', 'x70',
       'x71', 'x72', 'x73', 'x81', 'x82', 'x84', 'x87', 'x90', 'x92', 'x93',
       'x94', 'x97', 'x98', 'x100', 'cat__Friday', 'cat__Monday',
       'cat__Saturday', 'cat__Sunday', 'cat__Thursday', 'cat__Tuesday',
       'cat__Wednesday', 'cat__April', 'cat__August', 'cat__December',
       'cat__February', 'cat__January', 'cat__July', 'cat__June', 'cat__March',
       'cat__May', 'cat__November', 'cat__October', 'cat__September',
       'cat__allstate', 'cat__esurance', 'cat__farmers', 'cat__geico',
       'cat__progressive'],
      dtype='object')

### Scale data

In [60]:
scalerX = StandardScaler()
scalerX.fit(dfTrainClean[dropFeatures])
xTrainScaled = scalerX.transform(dfTrainClean[dropFeatures])
xTestScaled = scalerX.transform(dfTestClean[dropFeatures])

In [61]:
# logistic regression
logReg = LogisticRegression(C=7.7777,solver="lbfgs",max_iter=1111,tol=0.000003333, \
    multi_class="ovr",class_weight=None)

logReg.fit(xTrainScaled.astype(float),dfTrainClean["y"].astype(int))

LogisticRegression(C=7.7777, max_iter=1111, multi_class='ovr', tol=3.333e-06)

In [62]:
# neural networks
mlp = MLPClassifier(hidden_layer_sizes=(76,76,),random_state=None,alpha=0.1111, \
    solver='sgd',learning_rate="adaptive",activation="relu",n_iter_no_change=10, \
    tol=0.00001111,learning_rate_init=0.001111,max_iter=253,momentum=0.9111, \
    batch_size="auto",shuffle=True,warm_start=False,early_stopping=False,validation_fraction=0.1)

mlp.fit(xTrainScaled.astype(float),dfTrainClean["y"].astype(int))

C:\Users\TamLe\anaconda3\envs\sports\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (253) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(alpha=0.1111, hidden_layer_sizes=(76, 76),
              learning_rate='adaptive', learning_rate_init=0.001111,
              max_iter=253, momentum=0.9111, solver='sgd', tol=1.111e-05)

In [63]:
yPredTrainLog = logReg.predict(xTrainScaled.astype(float))
yPredTestLog = logReg.predict(xTestScaled.astype(float))
yProbTrainLog = logReg.predict_proba(xTrainScaled.astype(float))
yProbTestLog = logReg.predict_proba(xTestScaled.astype(float))

In [64]:
yPredTrainMLP = mlp.predict(xTrainScaled.astype(float))
yPredTestMLP = mlp.predict(xTestScaled.astype(float))
yProbTrainMLP = mlp.predict_proba(xTrainScaled.astype(float))
yProbTestMLP = mlp.predict_proba(xTestScaled.astype(float))

In [65]:
pd.DataFrame(yPredTrainLog).value_counts()

0    39238
1      762
dtype: int64

In [66]:
pd.DataFrame(dfTrain["y"]).value_counts()

y
0    34197
1     5803
dtype: int64

In [67]:
pd.DataFrame(yProbTrainLog).head(10)

,0,1
0,0.959971,0.040029
1,0.658801,0.341199
2,0.828818,0.171182
3,0.549976,0.450024
4,0.983968,0.016032
5,0.888512,0.111488
6,0.958273,0.041727
7,0.859887,0.140113
8,0.924804,0.075196
9,0.854813,0.145187


In [68]:
pd.DataFrame(yProbTestLog).head(15)

,0,1
0,0.979947,0.020053
1,0.972448,0.027552
2,0.740248,0.259752
3,0.822651,0.177349
4,0.986860,0.013140
5,0.942493,0.057507
6,0.725689,0.274311
7,0.940266,0.059734
8,0.673637,0.326363
9,0.376637,0.623363


In [69]:
pd.DataFrame(yProbTestLog).iloc[:,[1]].head(15)

,1
0,0.020053
1,0.027552
2,0.259752
3,0.177349
4,0.013140
5,0.057507
6,0.274311
7,0.059734
8,0.326363
9,0.623363


In [70]:
pd.DataFrame(yProbTestLog).iloc[:,[1]].to_csv("glmresults.csv",header=False,index=False)
pd.DataFrame(yProbTestMLP).iloc[:,[1]].to_csv("nonglmresults.csv",header=False,index=False)

In [71]:
print(len(yProbTestMLP))

10000


### Statsmodels

In [72]:
xInclConstant = sm.add_constant(dfTrainClean[dropFeatures])
model = sm.Logit(dfTrainClean["y"].astype(int),xInclConstant.astype(float))
# logistic regression
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.359918
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                40000
Model:                          Logit   Df Residuals:                    39933
Method:                           MLE   Df Model:                           66
Date:                Fri, 28 May 2021   Pseudo R-squ.:                  0.1308
Time:                        16:56:13   Log-Likelihood:                -14397.
converged:                       True   LL-Null:                       -16563.
Covariance Type:            nonrobust   LLR p-value:                     0.000
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const                3.5752   2.06e+06   1.73e-06      1.000   -4.04e+06    4.04e+06
x1                   0.0259      0.008      3.294      0.001       0.011       0.041
x2                   0.0954      0.022      4.407      0.000       0.053       0.138
x5                   0.0446      0.016      2.826      0.005       0.014       0.076
x7                 -74.1062      1.636    -45.290      0.000     -77.313     -70.899
x8                   0.1096      0.018      6.179      0.000       0.075       0.144
x13                  0.0194      0.009      2.108      0.035       0.001       0.037
x15                 -0.0205      0.014     -1.494      0.135      -0.047       0.006
x18                 -0.0094      0.005     -2.034      0.042      -0.018      -0.000
x19                 -0.0001   1.51e-05     -9.304      0.000      -0.000      -0.000
x20                 -0.0301      0.006     -4.903      0.000      -0.042      -0.018
x21                  0.0012      0.001      1.199      0.231      -0.001       0.003
x25                  0.0058      0.006      0.910      0.363      -0.007       0.018
x26                  0.0143      0.014      1.050      0.294      -0.012       0.041
x27                 -0.0004      0.000     -1.429      0.153      -0.001       0.000
x28                 -0.0973      0.012     -7.867      0.000      -0.122      -0.073
x29                 -0.0202      0.013     -1.574      0.115      -0.045       0.005
x31                 -0.5720      0.059     -9.740      0.000      -0.687      -0.457
x32                  0.0144      0.009      1.642      0.101      -0.003       0.032
x35                  0.0243      0.015      1.670      0.095      -0.004       0.053
x36                 -0.0115      0.010     -1.209      0.227      -0.030       0.007
x37                  0.0623      0.009      6.603      0.000       0.044       0.081
x40                 -0.1068      0.008    -13.138      0.000      -0.123      -0.091
x43                 -0.0168      0.015     -1.118      0.264      -0.046       0.013
x46                 -0.0549      0.012     -4.517      0.000      -0.079      -0.031
x47                 -0.1394      0.008    -16.839      0.000      -0.156      -0.123
x50              -9.406e-05   2.48e-05     -3.796      0.000      -0.000   -4.55e-05
x51                  0.0436      0.010      4.290      0.000       0.024       0.063
x56                 -0.0230      0.011     -2.158      0.031      -0.044      -0.002
x58                  0.0007      0.000      1.596      0.110      -0.000       0.002
x62                 -0.0533      0.013     -3.975      0.000      -0.080      -0.027
x63                  0.0444      0.013      3.485      0.000       0.019       0.069
x66                  0.0689      0.016      4.193      0.000       0.037       0.101
x67                 -0.0254      0.014     -1.867      0.062      -0.052       0.001
x68                  0.0145      0.003      4.265      0.000       0.008       0.021
x70        

In [73]:
# categorical_columns = ['Gender', 'House Type']
# for column in categorical_columns:
#     tempdf = pd.get_dummies(df[column], prefix=column)
#     df = pd.merge(
#         left=df,
#         right=tempdf,
#         left_index=True,
#         right_index=True,
#     )
#     df = df.drop(columns=column)
# print(df)

### AUC curve on train set
There is no dependent variable in test set.

In [119]:
yTrainClean = dfTrainClean["y"].to_numpy()
print(yTrainClean)
falsePositiveRate,truePositiveRate,threshold = metrics.roc_curve(yTrainClean,yProbTrainLog[:,1])

rocAucScore = metrics.roc_auc_score(yTrainClean,yProbTrainLog[:,1])
print("auc score: %s" % rocAucScore)
logLoss = log_loss(yTrainClean,yProbTrainLog[:,1])
print("log loss: %s" % logLoss)

[0. 1. 1. ... 0. 0. 0.]
0.7527958790394674
auc score: 0.7527958790394674
log loss: 0.35991762698348395


In [120]:
yTrainClean = dfTrainClean["y"].to_numpy()
print(yTrainClean)
falsePositiveRate,truePositiveRate,threshold = metrics.roc_curve(yTrainClean,yProbTrainMLP[:,1])

rocAucScore = metrics.roc_auc_score(yTrainClean,yProbTrainMLP[:,1])
print("auc score: %s" % rocAucScore)
logLoss = log_loss(yTrainClean,yProbTrainMLP[:,1])
print("log loss: %s" % logLoss)

[0. 1. 1. ... 0. 0. 0.]
auc score: 0.8318052312993567
log loss: 0.315821214868187
